# 第4章 朴素贝叶斯

1．朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布
$P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：

$$P(X,Y)＝P(Y)P(X|Y)$$

概率估计方法可以是极大似然估计或贝叶斯估计。

2．朴素贝叶斯法的基本假设是条件独立性，

$$\begin{aligned} P(X&=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ &=\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$


这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。

3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。

$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
 
将输入$x$分到后验概率最大的类$y$。

$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$

后验概率最大等价于0-1损失函数时的期望风险最小化。


模型：

- 高斯模型
- 多项式模型
- 伯努利模型

In [1]:
import numpy as np#用来处理数据或矩阵的库
import pandas as pd#解决数据分析任务，提供了大量能使我们快速便捷地处理数据的函数和方法
import matplotlib.pyplot as plt
%matplotlib inline
#当进行绘图时，或生成一个 figure 画布的时候，可以直接在你的python控制台里面生成图像
from sklearn.datasets import load_iris#导入数据集iris
from sklearn.model_selection import train_test_split#将数据划分训练集和测试集

from collections import Counter#计数器
import math#导入函数库

In [2]:
# data
def create_data():
    iris = load_iris()#载入数据集
    df = pd.DataFrame(iris.data, columns=iris.feature_names)#将iris数据集中的特征属性提取出来
    df['label'] = iris.target#将iris中的target作为df的一列
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]#给每一列数据添加标注
    data = np.array(df.iloc[:100, :])#读取数据前100行数据特征及标签
    # print(data)
    return data[:, :-1], data[:, -1]#返回去除最后一列data数据集的数据集，y取只有最后一列data数据集的数据集

In [3]:
X, y = create_data()#构建x、y数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
##拆分数据为训练集，和测试集，让测试数据占整个数据集的30%

In [4]:
X_test[0], y_test[0]

(array([5.7, 2.6, 3.5, 1. ]), 1.0)

参考：https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

## GaussianNB 高斯朴素贝叶斯

特征的可能性被假设为高斯

概率密度函数：
$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$

方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [5]:
class NaiveBayes:
    def __init__(self):
        self.model = None

    # 数学期望
    @staticmethod#def方法中没有调用self.的对象时使用
    def mean(X):#X是传入的特征向量
        return sum(X) / float(len(X))#返回x的数学期望

    # 标准差（方差）
    def stdev(self, X):
        avg = self.mean(X)#avg取x的数学期望
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))
        #返回x的方差：√[(x减avg的平方求和，x取n个)/n]

    # 概率密度函数
    def gaussian_probability(self, x, mean, stdev):#mean为数学期望，stdev为方差
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))#exponent取e^-(<x-mean>^2)/(2*stdev^2)
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent#返回概率密度：exponent/√(2π*stdev)
        
    # 处理X_train
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        #summaries取(i的数学期望,i方差)，i为train_data中特征向量的第i个特征组成
        return summaries

    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))#将y的数据集转成列表
        data = {label: [] for label in labels}#创建字典，每个键的值是列表
        for f, label in zip(X, y):#zip将对象相应位置元素打包成元组并返回元组组成的列表
            data[label].append(f)#将x,y中的数据遍历添加到data字典中
        self.model = {
            label: self.summarize(value)#帮助value内容概括和精确提取信息,提高可读性
            for label, value in data.items()#label和value取字典中的每对组成一个元组
        }
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):
        # summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}#设置一个空列表probabilities
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev)#根据求得的数学期望和标准差计算概率密度
        return probabilities

    # 类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        label = sorted(
            self.calculate_probabilities(X_test).items(),#对测试数据的概率组成进行排序，
            key=lambda x: x[-1])[-1][0]#待排序对象按最后一个维度的值进行排序
        return label

    def score(self, X_test, y_test):
        right = 0#初始化预测正确个数
        for X, y in zip(X_test, y_test):
            label = self.predict(X)#label取x的类别
            if label == y:#如果类别与y相同
                right += 1#正确个数+1

        return right / float(len(X_test))#返回测试集的正确率：正确个数/测试集个数

In [6]:
model = NaiveBayes()#建立朴素贝叶斯模型

In [7]:
model.fit(X_train, y_train)#求训练集的数学期望和标准差

'gaussianNB train done!'

In [8]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))#打印测试集为[4.4,3.2,1.3,0.2]时预测的类别

0.0


In [9]:
model.score(X_test, y_test)#返回测试集的正确率，结果显示1，预测正确

1.0

### scikit-learn实例

In [10]:
from sklearn.naive_bayes import GaussianNB#导入高斯朴素贝叶斯模型

In [11]:
clf = GaussianNB()
clf.fit(X_train, y_train)#求训练集的数学期望和标准差

GaussianNB()

In [12]:
clf.score(X_test, y_test)#返回测试集的正确率，结果显示正确率为1.0，预测正确

1.0

In [13]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])#打印测试集为[4.4,3.2,1.3,0.2]时预测的类别

array([0.])

In [14]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB #导入伯努利模型和多项式模型

## 第4章朴素贝叶斯法-习题

### 习题4.1
&emsp;&emsp;用极大似然估计法推出朴素贝叶斯法中的概率估计公式(4.8)及公式 (4.9)。

**解答：**  
**第1步：**证明公式(4.8)：$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k)}{N}$  
由于朴素贝叶斯法假设$Y$是定义在输出空间$\mathcal{Y}$上的随机变量，因此可以定义$P(Y=c_k)$概率为$p$。  
令$\displaystyle m=\sum_{i=1}^NI(y_i=c_k)$，得出似然函数：$$L(p)=f_D(y_1,y_2,\cdots,y_n|\theta)=\binom{N}{m}p^m(1-p)^{(N-m)}$$使用微分求极值，两边同时对$p$求微分：$$\begin{aligned}
0 &= \binom{N}{m}\left[mp^{(m-1)}(1-p)^{(N-m)}-(N-m)p^m(1-p)^{(N-m-1)}\right] \\
& = \binom{N}{m}\left[p^{(m-1)}(1-p)^{(N-m-1)}(m-Np)\right]
\end{aligned}$$可求解得到$\displaystyle p=0,p=1,p=\frac{m}{N}$  
显然$\displaystyle P(Y=c_k)=p=\frac{m}{N}=\frac{\displaystyle \sum_{i=1}^N I(y_i=c_k)}{N}$，公式(4.8)得证。

----

**第2步：**证明公式(4.9)：$\displaystyle P(X^{(j)}=a_{jl}|Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)}{\displaystyle \sum_{i=1}^N I(y_i=c_k)}$  
令$P(X^{(j)}=a_{jl}|Y=c_k)=p$，令$\displaystyle m=\sum_{i=1}^N I(y_i=c_k), q=\sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)$，得出似然函数：$$L(p)=\binom{m}{q}p^q(i-p)^{m-q}$$使用微分求极值，两边同时对$p$求微分：$$\begin{aligned}
0 &= \binom{m}{q}\left[qp^{(q-1)}(1-p)^{(m-q)}-(m-q)p^q(1-p)^{(m-q-1)}\right] \\
& = \binom{m}{q}\left[p^{(q-1)}(1-p)^{(m-q-1)}(q-mp)\right]
\end{aligned}$$可求解得到$\displaystyle p=0,p=1,p=\frac{q}{m}$  
显然$\displaystyle P(X^{(j)}=a_{jl}|Y=c_k)=p=\frac{q}{m}=\frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)}{\displaystyle \sum_{i=1}^N I(y_i=c_k)}$，公式(4.9)得证。

### 习题4.2
&emsp;&emsp;用贝叶斯估计法推出朴素贝叶斯法中的慨率估计公式(4.10)及公式(4.11)

**解答：**  
**第1步：**证明公式(4.11)：$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}$  
加入先验概率，在没有任何信息的情况下，可以假设先验概率为均匀概率（即每个事件的概率是相同的）。  
可得$\displaystyle p=\frac{1}{K} \Leftrightarrow pK-1=0\quad(1)$  
根据习题4.1得出先验概率的极大似然估计是$\displaystyle pN - \sum_{i=1}^N I(y_i=c_k) = 0\quad(2)$  
存在参数$\lambda$使得$(1) \cdot \lambda + (2) = 0$  
所以有$$\lambda(pK-1) + pN - \sum_{i=1}^N I(y_i=c_k) = 0$$可得$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}$，公式(4.11)得证。  

----

**第2步：**证明公式(4.10)：$\displaystyle P_{\lambda}(X^{(j)}=a_{jl} | Y = c_k) = \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k) + \lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + S_j \lambda}$   
根据第1步，可同理得到$$
P(Y=c_k, x^{(j)}=a_{j l})=\frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}$$  
$$\begin{aligned} 
P(x^{(j)}=a_{jl} | Y=c_k)
&= \frac{P(Y=c_k, x^{(j)}=a_{j l})}{P(y_i=c_k)} \\
&= \frac{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}}{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}} \\
&= (\lambda可以任意取值，于是取\lambda = S_j \lambda) \\
&= \frac{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}}{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K S_j \lambda}} \\ 
&= \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda} (其中\lambda = S_j \lambda)\\
&= \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k) + \lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + S_j \lambda}
\end{aligned} $$  
公式(4.11)得证。

----
参考代码：https://github.com/wzyonggege/statistical-learning-method

本文代码更新地址：https://github.com/fengdu78/lihang-code

习题解答：https://github.com/datawhalechina/statistical-learning-method-solutions-manual

中文注释制作：机器学习初学者公众号：ID:ai-start-com

配置环境：python 3.5+

代码全部测试通过。
![gongzhong](../gongzhong.jpg)